In [68]:
using LowLevelFEM
import LowLevelFEM as FEM
gmsh.initialize()

In [69]:
gmsh.open("body1-2D.geo")

Info    : Reading 'body1-2D.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000508465s, CPU 0.000509s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0113788s, CPU 0.009196s)
Info    : 98 nodes 198 elements
Info    : Done reading 'body1-2D.geo'


In [70]:
mat = FEM.material("body")
problem = FEM.Problem([mat], type=:PlaneHeatConduction)

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 16)


LowLevelFEM.Problem("body1-2D", :PlaneHeatConduction, 2, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 98)

In [71]:
supp = FEM.temperatureConstraint("left", T=0)
load = FEM.heatFlux("right", qn=-1)

("right", 1, 0, 0)

In [72]:
T0 = FEM.initialTemperature(problem, "body", T=0)

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0], Int64[], 1, :T)

In [73]:
FEM.initialTemperature!(problem, "left", T0, T=0)
T0

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0], Int64[], 1, :T)

In [74]:
K = FEM.heatConductionMatrix(problem)
C = FEM.heatCapacityMatrix(problem)
q = FEM.heatFluxVector(problem, [load])

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; 0.062499999999999924; … ; 0.0; 0.0;;], Float64[], Int64[], 1, :qn)

In [75]:
FEM.applyBoundaryConditions!(problem, K, C, q, [supp])

In [76]:
λmax = FEM.largestEigenValue(K, C)

28634.166285282736

In [77]:
ϑ = 0.33
T = FEM.FDM(K, C, q, T0, 200 * (2 / ((1 - 2ϑ) * λmax)), 10 * 2 / ((1 - 2ϑ) * λmax), ϑ=ϑ)

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0060297789879445165 … -5.641254670169754 10.156487808931942; … ; 0.0 1.6743417968164684e-6 … -0.021679854915272145 0.05736439147570544; 0.0 0.00038000429880581215 … 0.21748580922588828 -0.8924961347251508], [0.0, 0.0020543126286864714, 0.004108625257372943, 0.006162937886059414, 0.008217250514745885, 0.010271563143432357, 0.012325875772118829, 0.0143801884008053, 0.01643450102949177, 0.01848881365817824, 0.02054312628686471, 0.02259743891555118, 0.02465175154423765, 0.02670606417292412, 0.02876037680161059, 0.03081468943029706, 0.032869002058983535, 0.03492331468767001, 0.03697762731635648, 0.039031939945042955], Int64[], 20, :T)

In [78]:
ϑ = 0.66
T = FEM.FDM(K, C, q, T0, 0.15, 0.01, ϑ=ϑ)

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.009997131200716583 … 0.02195142944671445 0.022026122044932768; … ; 0.0 0.0006955028498644957 … 0.0056738507269208415 0.005703532649398027; 0.0 0.004299019693876794 … 0.01639374619105087 0.016462740381772208], [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.07, 0.08, 0.09, 0.09999999999999999, 0.10999999999999999, 0.11999999999999998, 0.12999999999999998, 0.13999999999999999], Int64[], 15, :T)

In [79]:
FEM.showDoFResults(problem, T, :T, visible=true)

0

In [80]:
q1 = FEM.solveHeatFlux(problem, T)

LowLevelFEM.VectorField([[0.0 -0.8737295094799341 … -0.9950319310459341 -0.996423284463112; 0.0 0.0045167319574253295 … 4.5151742416177365e-5 3.3471992803707495e-5; … ; 0.0 -0.8737295094799341 … -0.9950319310459341 -0.996423284463112; 0.0 0.0045167319574253295 … 4.5151742416177365e-5 3.3471992803707495e-5], [0.0 -0.8752832765651961 … -0.9950336078296276 -0.9964242720440206; 0.0 0.006883298036587937 … 4.77056769450368e-5 3.497619217626702e-5; … ; 0.0 -0.8752832765651961 … -0.9950336078296276 -0.9964242720440206; 0.0 0.006883298036587937 … 4.77056769450368e-5 3.497619217626702e-5], [0.0 -0.8519119019378492 … -0.9948052809402799 -0.9962591186700933; 0.0 -0.021856241883897207 … -0.00019636914842855076 -0.00013998047680337418; … ; 0.0 -0.8519119019378492 … -0.9948052809402799 -0.9962591186700933; 0.0 -0.021856241883897207 … -0.00019636914842855076 -0.00013998047680337418], [0.0 -0.12874648956120274 … -0.9817238415478617 -0.9867701484671489; 0.0 -0.0038867366066517965 … -0.000142825316497736

In [81]:
FEM.showHeatFluxResults(problem, q1, :qvec)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


1

In [82]:
q2 = FEM.solveHeatFlux(problem, T, DoFResults=true)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 -0.10867379353969966 … -0.9809659091686 -0.9862212190272407; 0.0 0.0004479021629753301 … 1.7803541702528583e-5 1.289491349776642e-5; … ; 0.0 -0.6669871496499238 … -0.9926187961925689 -0.9946654638767273; 0.0 -0.0011124294887973455 … -7.173390802403534e-6 -5.034886551118223e-6], [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.07, 0.08, 0.09, 0.09999999999999999, 0.10999999999999999, 0.11999999999999998, 0.12999999999999998, 0.13999999999999999], Int64[], 15, :q2D)

In [83]:
FEM.showDoFResults(problem, q2, :qvec)

2

In [84]:
q3 = FEM.elementsToNodes(problem, q1)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 -0.10867379353969966 … -0.9809659091686 -0.9862212190272407; 0.0 0.0004479021629753301 … 1.7803541702528583e-5 1.289491349776642e-5; … ; 0.0 -0.6669871496499238 … -0.9926187961925689 -0.9946654638767273; 0.0 -0.0011124294887973455 … -7.173390802403534e-6 -5.034886551118223e-6], [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.07, 0.08, 0.09, 0.09999999999999999, 0.10999999999999999, 0.11999999999999998, 0.12999999999999998, 0.13999999999999999], Int64[], 15, :q2D)

In [85]:
FEM.showDoFResults(problem, q3, :qvec)

3

In [86]:
err1 = FEM.fieldError(problem, q1)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 0.0004479021629753485 … 1.7803541702554604e-5 1.2894913497873972e-5; 0.0 0.0004479021629753301 … 1.7803541702528583e-5 1.289491349776642e-5; … ; 0.0 0.1132033221401278 … 0.0014187127541194025 0.0010355757458751572; 0.0 0.048599625420262346 … 0.0006229325653385644 0.0004543881746332445], [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.07, 0.08, 0.09, 0.09999999999999999, 0.10999999999999999, 0.11999999999999998, 0.12999999999999998, 0.13999999999999999], Int64[], 15, :q2D)

In [87]:
FEM.showDoFResults(problem, err1, :vector)

4

In [88]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [89]:
gmsh.finalize()